In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [7]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|           countries|      false|
| tabd_db|countryinternetusers|      false|
| tabd_db|dailyinternetusag...|      false|
| tabd_db|    internetjoindate|      false|
| tabd_db|listleastdevelope...|      false|
| tabd_db|listofcountriesby...|      false|
| tabd_db|parentawarenessof...|      false|
+--------+--------------------+-----------+



In [5]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms (
        region VARCHAR(45),
        platform VARCHAR(45),
        percentage_cyberbullying INT
    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/P_ParentAwarenessOfCyberBullyingPlatforms/'
    """
)

# tblproperties('skip.header.line.count'='1')
# can be used for csvs with header
# but spark sql cannot understand that at the moment, while reading the data using sql queries
# so just when creating hive tables backed up by csvs, avoid headers
# in the project we wont use hive text tables, so all good.

DataFrame[]

In [5]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|countryinternetusers|      false|
| tabd_db|parentawarenessof...|      false|
+--------+--------------------+-----------+



In [6]:
# Let's look into HDFS

In [7]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms
    """
).show()

+--------+------------------------+------+
|platform|percentage_cyberbullying|region|
+--------+------------------------+------+
+--------+------------------------+------+



In [8]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms
    """
).toPandas()

,col_name,data_type,comment
0,platform,varchar(45),None
1,percentage_cyberbullying,int,None
2,region,varchar(45),None
3,# Partition Information,,
4,# col_name,data_type,comment
5,region,varchar(45),None
6,,,
7,# Detailed Table Information,,
8,Database,tabd_db,
9,Table,parentawarenessofcyberbullyingplatforms,


In [9]:
# Let's put the files into HDFS

In [10]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms
    """
).toPandas()

,platform,percentage_cyberbullying,region


In [11]:
# recover partitions is needed so that the Hive Metastore (Catalog)
# is updated. Otherwise Hive and the querying engines do not know
# that there are new parittions in the partitioned table.
spark.catalog.recoverPartitions("tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms")

spark.sql(
    """
    SELECT *
    FROM tabd_db.P_ParentAwarenessOfCyberBullyingPlatforms
    """
).show()

+--------+------------------------+------+
|platform|percentage_cyberbullying|region|
+--------+------------------------+------+
+--------+------------------------+------+



In [12]:
spark.stop()